In [1]:
from __future__ import division
import os
import cv2
import numpy as np
import sys
import pickle
from optparse import OptionParser
import time
from keras_frcnn import config
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras_frcnn import roi_helpers
import keras_frcnn.vgg as nn

Using TensorFlow backend.


In [64]:
sys.setrecursionlimit(40000)

config_filename="./config_file/config.pickle"

with open(config_filename, 'rb') as f_in:
    C = pickle.load(f_in)

class_mapping = C.class_mapping
C.model_path = './config_file/model_frcnn_.hdf5'
if 'bg' not in class_mapping:
    class_mapping['bg'] = len(class_mapping)

class_mapping = {v: k for k, v in class_mapping.items()}
print(class_mapping)
class_to_color = {class_mapping[v]: np.random.randint(0, 255, 3) for v in class_mapping}

{0: 'person', 1: 'bicycle', 2: 'dog', 3: 'chair', 4: 'pottedplant', 5: 'motorbike', 6: 'boat', 7: 'car', 8: 'bird', 9: 'bottle', 10: 'diningtable', 11: 'horse', 12: 'sofa', 13: 'tvmonitor', 14: 'cat', 15: 'sheep', 16: 'cow', 17: 'aeroplane', 18: 'bus', 19: 'train', 20: 'bg'}


In [2]:
from moviepy.editor import VideoFileClip # Import Video Moudule

video_path = './dataset/youtube.mp4'
clip = VideoFileClip(video_path)
new_video = video_path.split("/")
new_video[-1] = "detected_"+new_video[-1]
new_video = "/".join(new_video)

Imageio: 'ffmpeg-linux64-v3.3.1' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-linux64-v3.3.1 (43.8 MB)
Downloading: 8192/45929032 bytes (0.040960/45929032 bytes (0.1%212992/45929032 bytes (0.5385024/45929032 bytes (0.8491520/45929032 bytes (1.1638976/45929032 bytes (1.4786432/45929032 bytes (1.7950272/45929032 bytes (2.11114112/45929032 bytes (2.4%1294336/45929032 bytes (2.8%1474560/45929032 bytes (3.2%1490944/45929032 bytes (3.2%1671168/45929032 bytes (3.6%1687552/45929032 bytes (3.7%1884160/45929032 bytes (4.1%1933312/45929032 bytes (4.2%2260992/45929032 bytes (4.9%2424832/45929032 bytes (5.3%2588672/45929032 bytes (5.6%2752512/45929032 bytes (6.0%2916352/45929032 bytes (6.3%3096576/45929032 bytes (6.7%3276800/45929032 bytes (7.1%3457024/45929032 bytes (7.5%3653632/45929032 bytes (8.0%3833856/45929032 bytes (8.3%4030464/45929032 bytes (8.8%4227072/45929032 bytes (9.2%4423680/45929032 byte

ModuleNotFoundError: No module named 'requests'

In [21]:
def format_img_size(img, C):
    """ formats the image size based on config """
    img_min_side = float(C.im_size)
    (height, width ,_) = img.shape

    if width <= height:
        ratio = img_min_side/width
        new_height = int(ratio * height)
        new_width = int(img_min_side)
    else:
        ratio = img_min_side/height
        new_width = int(ratio * width)
        new_height = int(img_min_side)
    img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
    return img, ratio

def format_img_channels(img, C):
    """ formats the image channels based on config """
    img = img[:, :, (2, 1, 0)]
    img = img.astype(np.float32)
    img[:, :, 0] -= C.img_channel_mean[0]
    img[:, :, 1] -= C.img_channel_mean[1]
    img[:, :, 2] -= C.img_channel_mean[2]
    img /= C.img_scaling_factor
    img = np.transpose(img, (2, 0, 1))
    img = np.expand_dims(img, axis=0)
    return img

def format_img(img, C):
    """ formats an image for model prediction based on config """
    img, ratio = format_img_size(img, C)
    img = format_img_channels(img, C)
    return img, ratio

# Method to transform the coordinates of the bounding box to its original size
def get_real_coordinates(ratio, x1, y1, x2, y2):

    real_x1 = int(round(x1 // ratio))
    real_y1 = int(round(y1 // ratio))
    real_x2 = int(round(x2 // ratio))
    real_y2 = int(round(y2 // ratio))

    return (real_x1, real_y1, real_x2 ,real_y2)

In [22]:
img_input = Input(shape=(None, None, 3))
roi_input = Input(shape=(C.num_rois, 4))
feature_map_input = Input(shape=(None, None, 512))
shared_layers = nn.nn_base(img_input, trainable=True)

In [23]:
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn_layers = nn.rpn(shared_layers, num_anchors)

classifier = nn.classifier(feature_map_input, roi_input, C.num_rois, nb_classes=len(class_mapping), trainable=True)

model_rpn = Model(img_input, rpn_layers)
model_classifier_only = Model([feature_map_input, roi_input], classifier)

model_classifier = Model([feature_map_input, roi_input], classifier)

print('Loading weights from {}'.format(C.model_path))
result1=model_rpn.load_weights(C.model_path, by_name=True)
result2 = model_classifier.load_weights(C.model_path, by_name=True)

model_rpn.compile(optimizer='sgd', loss='mse')
model_classifier.compile(optimizer='sgd', loss='mse')

Loading weights from ./config_file/model_frcnn.hdf5


In [61]:
all_imgs = []

bbox_threshold = 0.8

visualise = True

for img in clip.iter_frames():

    st = time.time()
    X, ratio = format_img(img, C)

    if K.image_dim_ordering() == 'tf':
        X = np.transpose(X, (0, 2, 3, 1))

    # get the feature maps and output from the RPN
    [Y1, Y2, F] = model_rpn.predict(X)


    R = roi_helpers.rpn_to_roi(Y1, Y2, C, K.image_dim_ordering(), overlap_thresh=0.7)

    # convert from (x1,y1,x2,y2) to (x,y,w,h)
    R[:, 2] -= R[:, 0]
    R[:, 3] -= R[:, 1]

    # apply the spatial pyramid pooling to the proposed regions
    bboxes = {}
    probs = {}

    for jk in range(R.shape[0]//C.num_rois + 1):
        ROIs = np.expand_dims(R[C.num_rois*jk:C.num_rois*(jk+1), :], axis=0)
        if ROIs.shape[1] == 0:
            break

        if jk == R.shape[0]//C.num_rois:
            #pad R
            curr_shape = ROIs.shape
            target_shape = (curr_shape[0],C.num_rois,curr_shape[2])
            ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
            ROIs_padded[:, :curr_shape[1], :] = ROIs
            ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
            ROIs = ROIs_padded

        [P_cls, P_regr] = model_classifier_only.predict([F, ROIs])

        for ii in range(P_cls.shape[1]):

            if np.max(P_cls[0, ii, :]) < bbox_threshold or np.argmax(P_cls[0, ii, :]) == (P_cls.shape[2] - 1):
                continue

            cls_name = class_mapping[np.argmax(P_cls[0, ii, :])]

            if cls_name not in bboxes:
                bboxes[cls_name] = []
                probs[cls_name] = []

            (x, y, w, h) = ROIs[0, ii, :]

            cls_num = np.argmax(P_cls[0, ii, :])
            try:
                (tx, ty, tw, th) = P_regr[0, ii, 4*cls_num:4*(cls_num+1)]
                tx /= C.classifier_regr_std[0]
                ty /= C.classifier_regr_std[1]
                tw /= C.classifier_regr_std[2]
                th /= C.classifier_regr_std[3]
                x, y, w, h = roi_helpers.apply_regr(x, y, w, h, tx, ty, tw, th)
            except:
                pass
            bboxes[cls_name].append([C.rpn_stride*x, C.rpn_stride*y, C.rpn_stride*(x+w), C.rpn_stride*(y+h)])
            probs[cls_name].append(np.max(P_cls[0, ii, :]))

    all_dets = []

    for key in bboxes:
        bbox = np.array(bboxes[key])

        new_boxes, new_probs = roi_helpers.non_max_suppression_fast(bbox, np.array(probs[key]), overlap_thresh=0.5)
        for jk in range(new_boxes.shape[0]):
            (x1, y1, x2, y2) = new_boxes[jk,:]

            (real_x1, real_y1, real_x2, real_y2) = get_real_coordinates(ratio, x1, y1, x2, y2)

            cv2.rectangle(img,(real_x1, real_y1), (real_x2, real_y2), (int(class_to_color[key][0]), int(class_to_color[key][1]), int(class_to_color[key][2])),2)

            textLabel = '{}: {}'.format(key,int(100*new_probs[jk]))
            all_dets.append((key,100*new_probs[jk]))

            (retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_COMPLEX,1,1)
            textOrg = (real_x1, real_y1-0)

            cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (0, 0, 0), 2)
            cv2.rectangle(img, (textOrg[0] - 5,textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (255, 255, 255), -1)
            cv2.putText(img, textLabel, textOrg, cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 1)
    

    print('Elapsed time = {}'.format(time.time() - st))
    print(all_dets)
    all_imgs.append(img)
    

Elapsed time = 0.41966676712036133
[('person', 99.69702959060669)]
Elapsed time = 0.39853382110595703
[('person', 99.05735850334167)]
Elapsed time = 0.39955782890319824
[('person', 96.42081260681152)]
Elapsed time = 0.415708065032959
[('person', 98.77479076385498)]
Elapsed time = 0.39871859550476074
[('person', 98.89511466026306)]
Elapsed time = 0.4101417064666748
[('person', 97.0376193523407)]
Elapsed time = 0.39165711402893066
[('person', 97.78234958648682)]
Elapsed time = 0.40453648567199707
[('person', 94.13650035858154)]
Elapsed time = 0.3895304203033447
[('person', 91.09945297241211)]
Elapsed time = 0.39266395568847656
[('person', 90.36232829093933)]
Elapsed time = 0.3907461166381836
[('person', 95.46053409576416)]
Elapsed time = 0.3998093605041504
[('person', 95.06790041923523)]
Elapsed time = 0.3977174758911133
[('person', 92.5082266330719)]
Elapsed time = 0.3842184543609619
[('person', 98.86730313301086)]
Elapsed time = 0.38668251037597656
[('person', 98.82845878601074)]
Elaps

In [62]:
from moviepy.editor import ImageSequenceClip
new_clips = ImageSequenceClip(all_imgs, fps=24)
new_clips.write_videofile(new_video, fps=24)

[MoviePy] >>>> Building video ./dataset/detected_youtube.mp4
[MoviePy] Writing video ./dataset/detected_youtube.mp4


100%|██████████| 4440/4440 [00:16<00:00, 268.21it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./dataset/detected_youtube.mp4 

